# Hoeffding Tree first split

In [1]:
import math
from collections import Counter
from collections import defaultdict


class HoeffdingTree:
    def __init__(self, Nmin, delta, R, logfunc, verbose=True):
        self.Nmin = Nmin
        self.delta = delta
        self.R = R
        self.logfunc = logfunc
        self.verbose = verbose

    def log_debug(self, *msg):
        if self.verbose:
            print(*msg)

    def calc_hoeffding_bound(self, i):
        print(f"hb = sqrt({self.R}^2*ln(1/{self.delta})/2*{i}) = {math.sqrt((self.R ** 2 * math.log(1. / self.delta)) / (2 * i))}")
        return (
            math.sqrt((self.R ** 2 * math.log(1. / self.delta)) / (2 * i))
        )

    def calc_prior_entropy(self, labels):
        c = Counter(labels)
        entropy = 0
        print(f"\nentropy(D_{len(labels)})= -(", end = '')
        for label in set(labels):
            p = c[label] / len(labels)
            entropy += p * self.logfunc(p)
            print(f"{p}*{self.logfunc(p)} +", end = '')
        print(f")={-entropy}")
        return -entropy

    def calc_total_attr_entropy(self, kv, i):
        #self.log_debug(kv.keys())
        weighted_sum = 0
        print(f"Entropy({kv.keys()})= ", end = '')
        for item in kv.keys():
            entropy = 0
            count = sum(kv[item].values())
            for label_count in kv[item].values():
                p = label_count / count
                entropy += p * self.logfunc(p)
            weighted_sum += count / i * (-entropy)
            print(f"{count}/{i} * -({entropy})+", end = '')
            #self.log_debug('entropy for', item, ": ", -entropy)
        print(f"={weighted_sum}")
        # self.log_debug('weighted_sum:', weighted_sum)
        return weighted_sum

    def fit(self, data_columns, label_column):
        total_len = len(next(iter(data_columns.values())))
        for i in range(1, total_len + 1):
            if i % self.Nmin != 0:
                continue

            print(f"n={i}")
            #self.log_debug(i)

            infogains = []

            for attr_name, col in data_columns.items():
                kv = defaultdict(lambda: defaultdict(int))
                for (item, label) in zip(col[:i], label_column[:i]):
                    kv[item][label] += 1

                prior_entropy = self.calc_prior_entropy(label_column[:i])
                total_attr_entropy = self.calc_total_attr_entropy(kv, i)

                infogain = prior_entropy - total_attr_entropy
                print(f"infogain({attr_name})={prior_entropy}-{total_attr_entropy}={infogain}")
                #self.log_debug(
                #    'infogain:',
                #    prior_entropy,
                #    "-",
                #    total_attr_entropy,
                #    '= ',
                #    infogain
                #)
                infogains.append((infogain, attr_name))

            infogains = sorted(infogains)
            print('All infogains:', infogains)
            eps = self.calc_hoeffding_bound(i)
            if infogains[-1][0] - infogains[-2][0] > eps:
                print('Split on:', infogains[-1][1])
            else:
                print("Information Gain too small, no split.")
            print('')

In [2]:
Nmin = 2
delta = 0.2
R = 1
grace = 2
logfunc = math.log2
tree = HoeffdingTree(Nmin, delta, R, logfunc)
data_columns = {
    'time': ['1-2', '2-7', '>7', '1-2', '>7', '1-2', '2-7', '2-7'],
    'gender': ['m', 'm', 'f', 'f', 'm', 'm', 'f', 'm'],
    'area': ['urban', 'rural', 'rural', 'rural', 'rural', 'rural', 'urban', 'urban']
}
label_columns = [
    ['low', 'high', 'low', 'high', 'high', 'high', 'low', 'low']
]
tree.fit(data_columns, label_columns[0])

n=2

entropy(D_2)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['1-2', '2-7']))= 1/2 * -(0.0)+1/2 * -(0.0)+=0.0
infogain(time)=1.0-0.0=1.0

entropy(D_2)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['m']))= 2/2 * -(-1.0)+=1.0
infogain(gender)=1.0-1.0=0.0

entropy(D_2)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['urban', 'rural']))= 1/2 * -(0.0)+1/2 * -(0.0)+=0.0
infogain(area)=1.0-0.0=1.0
All infogains: [(0.0, 'gender'), (1.0, 'area'), (1.0, 'time')]
hb = sqrt(1^2*ln(1/0.2)/2*2) = 0.6343181205897598
Information Gain too small, no split.

n=4

entropy(D_4)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['1-2', '2-7', '>7']))= 2/4 * -(-1.0)+1/4 * -(0.0)+1/4 * -(0.0)+=0.5
infogain(time)=1.0-0.5=0.5

entropy(D_4)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['m', 'f']))= 2/4 * -(-1.0)+2/4 * -(-1.0)+=1.0
infogain(gender)=1.0-1.0=0.0

entropy(D_4)= -(0.5*-1.0 +0.5*-1.0 +)=1.0
Entropy(dict_keys(['urban', 'rural']))= 1/4 * -(0.0)+3/4 * -(-0.9182958340544896)+=0.6887218755408672
infog